# SIXT33N Project
## Phase 5: PCA/Classificiation - Voice Commands

### EE 16B: Designing Information Devices and Systems II, Fall 2018

Written by Nathaniel Mailoa and Emily Naviasky (2016). Updated by Julian Chan (2018).

nmailoa@berkeley.edu &emsp; enaviasky@berkeley.edu &emsp; julianchan0928@berkeley.edu

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

## Table of Contents

* [Introduction](#intro)
* [Part 1: Data Collection](#part1)
* [Part 2: Principal Component Analysis](#part2)
* [Part 3: Finding Cluster Centers](#part3)
* [Part 4: Classifying Test Data](#part4)

<a id='intro'></a>
## Introduction

SIXT33N is an obedient little robot that will follow the directions that you tell it. There are four moves that SIXT33N can make: move straight, move straight slowly, turn right, and turn left. However, SIXT33N does not speak human languages, and some words, like "left" and "right", sound very similar (a strong single syllable), while other words are easy to distinguish. Your job in this phase is to find four command words that are easy for SIXT33N to tell apart (consider syllables and intonation).

For phase 3, you will develop the PCA classifier that allows SIXT33N to tell the difference between the four commands. You will examine several different words, and determine which ones will be easiest to sort by PCA.

### Side Note: Datasets in Machine Learning Applications
It is common practice, especially in machine learning applications, to split a dataset into a training set and a smaller test set (some common ratios for train:test are 80:20 or 70:30) when trying to make data-driven predictions/decisions. In this lab, we will collect data and split our dataset into 70% training data and 30% test data. 

### Overview of Classification Procedure
Once you have some sample data collected, we will:
1. Split our data into 2 sets: train_data and test_data
2. Perform PCA and look at how well it separates the train_data 
3. Once you have a set of four words that you like, you will compute the means for each of those four words in the PCA basis. We will classify each word according to which mean it is closest in Euclidean distance to. 
4. To see how well our classifier does on data it has never seen before (this is called generalization in machine learning), we will project test_data onto the same PCA basis as train_data, and find the mean that is closest in Euclidean distance to each data point. 
5. When you (and your GSI) are satisfied with the classifier's accuracy, you will port the classifier into the Launchpad code in Energia.

<b>
The goals of this phase are as follows:
- Generate envelope and utilize threshold to get snippets
- PCA + Classifier (4 commands)
- Check accuracy
- PCA projection on Launchpad
</b>

The checkpoints (marked **<span style="color:green">green</span>** in the Notebook) are:
- Checkpoint 1: First pass through PCA with training data; GSI feedback
- Checkpoint 2: Classification target met in Python using test data

Note that all of this is considered a single graded lab.

<a id='part1'></a>
## <span style="color:blue">Part 1: Data Collection</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB

When humans distinguish words, they listen for temporal and frequency differences to determine what is being said. However, SIXT33N does not have the memory or the processing power to distinguish words nearly as well as our human brains, so we will have to choose much simpler features for SIXT33N to look at (syllables, intonation, magnitude).

When you think of speech signals, you might notice that the shape of the speech wave is a very distinctive part of each word. Taking just the shape of the magnitude of a signal is called enveloping, exemplified in the image below. So, we want to do some filtering to retrieve the envelope of the audio signal. We train the PCA off of just this envelope and build a classifier to classify new data points.

<center>
<img width="400px" src="images/proj-envelope.png">
</center>

<b>Keeping in mind that the words that look most different have different shapes (or different amplitudes varied over time), brainstorm four or five words that you think will sort well. Consider syllables, intonation, and length of the word.</b>

**<span style="color:red">What words are you going to try? Why?</span>** 

Firstly, upload the sketch <b>`collect-data-envelope.ino`</b> to your Launchpad. This sketch gathers ADC samples every 0.35ms, and streams the data back to the PC.

Next, hook up your front end circuit.<b> Make sure to disconnect the 5V jumper from the Launchpad</b>, or you will power the Launchpad from the power supply as well as from the USB cord. 

<b>Tip: To not lose the jumper, only connect the jumper on one pin so it stays on your launchpad without powering it and creating a connection. Remember to put it back at the end of the lab. The pictures below shows the original configuration of the 5V jumper and modified one for this lab</b> 

<center>
<img width="400px" src="images/msp-jumpers.png">
<img width="400px" src="images/5vjumper.jpg">
</center>


Do not power up your circuit for now, we will connect the appropriate Launchpad pins to your circuit first:
- P6.0 to the microphone front end circuit output
- 3.3V pin to the 3.3V power rail of the breadboard (in particular to supply the buffer op-amp)
- 5V pin to the 5V row where your MicBoard V_DD is located
- GND pin to the ground rail of the breadboard

You can keep your MSP plugged via USB as long as <b>YOUR JUMPER IS DISCONNECTED</b>

Next, use the bench power supply to provide 5V to the circuit. **<span color='red'>DO NOT FORGET TO SET THE CURRENT LIMIT.</span>** <b>Before you start recording, use the oscilloscope to probe the output of the microphone circuit. Make sure the waveform averages to 1.65V (halfway between 0V and 3.3V) and the amplitude is large enough.</b> Make a noise at the microphone; you should see the signal change to reflect the sound you just made. If you are close enough or loud enough, you should be able to get the peak-to-peak amplitude of your signal all the way up to 3.3V.

<center>
<img width="400px" src="images/proj-waveform.png">
</center>

"Good" Audio data has a high signal to noise ratio. Recording words while far away from the microphone may cause your intended word to blend in with background noise. However "oversaturation" of the audio signal (speaking too loudly and/or too closely into the mic) will distort the signal (Why?). You can use the oscilloscope on the front-end output to test for over/under saturation.

You should already have <b>`collect-data-envelope.ino`</b> uploaded to your Launchpad. This sketch will turn on the red Launchpad LED to show that it is recording. The Launchpad will record 2 seconds of audio at a time, sampled every 0.35ms. 

<b>The red LED on the launch pad is like a recording room. When the red light goes on, the Launchpad is recording. Say the word you want to record before the red LED turns off.</b>

To make your life easier, pronounce the words consistently. Try <b>five or six words</b> that you think will classify well. The Launchpad will apply an enveloping function (discussed later) which reduces the data from several thousand samples to 172 samples. 

The Launchpad has limited memory and will delete the enveloped data as soon as the red light flashes again. To record the data permanently, the Launchpad must be connected to a PC by the USB, allowing it to transfer its data

To transfer the data in the PC, make sure the Launchpad is connected, then run:

<b>`python collect-data-envelope.py log.csv`</b>

Now, each time the light flashes and a sample is collected by the Launchpad, it will be written into to some file called `log.csv`. You might want to probe the output and watch the scope while you collect. After you collect, a few test words, check the `log.csv` and make sure that it looks like a sound wave and is not just full of zeros. It might help to plot the data on excel to make sure.

Now the Launchpad will send the audio data to the PC in addition to collecting and enveloping it. The python script will write into an excel file called `log.csv`, one enveloped sample per row. After collection, make sure the tables are not filled with zeroes.


<b>Collect around 30 good samples for each of your four to five words</b>, and be sure to save them to different .csv files with descriptive names. You will collect more once you settle on the four words that sort well enough.

Just in case: To write to different .csv files, just change `log.csv` to `TheWordYouWant.csv` in <b>`python collect-data-envelope.py log.csv`</b>.

#### For your Consideration:

Once you have your four or more words collected, you can move onto the PCA classification below. You may realize in the next section that one or two of your words are not sorting quite as well as you would like. Don't be afraid to come back to this section and try collecting different words based on what you have learned makes a word sortable. 


**<span style="color:red"> Summary TO DO for part 1: </span>** 
- **<span style="color:red">Keep MSP powered via USB</span>**
- **<span style="color:red">Disconnect 5V jumper from Launchpad</span>**
- **<span style="color:red">Connect P6.0 to the microphone front end circuit output</span>**
- **<span style="color:red">Connect 3.3V pin of the launchpad to the 3V power rail of the breadboard (in particular to supply the buffer op-amp) </span>**
- **<span style="color:red">Connect the 5V pin of the launchpad to V_DD of micboard and supply 5V from the power supply (remember to set the current limit!)</span>**
- **<span style="color:red">Connect the GND pin of the launchpad to the ground rail of the breadboard</span>**
- **<span style="color:red">Upload `collect-data-envelope.ino` onto launchpad</span>**
- **<span style="color:red">Run `python collect-data-envelope.py log.csv` on your PC to collect data into `log.csv`</span>**
- **<span style="color:red">Record 30 samples for each of your 4 to 5 words (Saying your word each time the red light flashes)</span>**

<a id='part2'></a>
## <span style="color:blue">Part 2: Principal Component Analysis</span>

Before we can use the recorded data for PCA, we must first process the data. It is not necessary for you to understand the enveloping function well enough to implement it (since we have already done it for you), but just in case you are curious the enveloping function is described in the following pseudocode:

<code><b>Enveloping function</b>
Divide the whole signal to a block of 16 samples
For each chunk:
    Find the mean of the chunk
    Subtract each sample by the mean
    Find the sum of the absolute value of each sample
</code>

What you really need to know, however, is what the enveloped signal looks like for each word. Spend a little time looking at the data you just collected in the python plots below.

First, load the recorded data from the csv files.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import csv
%matplotlib inline

In [ ]:
def read_csv(filename):
    data = []
    with open(filename, 'r') as csvfile:
        r = csv.reader(csvfile, delimiter=' ')
        for row in r:
            data.append([float(i) for i in row[0].split(',')])
    return np.array(data)

def train_test_split(data, split_ratio=0.7):
    """
    Splits data into training and test set according to the split_ratio.
    
    Arguments:
        data: dataset as a numpy array
        split_ratio: fraction of dataset to split as training data (must be between 0 and 1)
    
    Returns:
        Training Data (size = split_ratio * size of original dataset)
        Test Data (size = (1 - split_ratio) * size of original dataset)
    """
    train_data, test_data = data[:int(split_ratio*len(data)),:], data[int(split_ratio*len(data)):,:]
    
    return train_data, test_data

In [ ]:
# Load data from csv
# YOUR CODE HERE
word1_raw = read_csv("")
word2_raw = read_csv("")
word3_raw = read_csv("")
word4_raw = read_csv("")

# Split the data into training and test set
train_test_split_ratio = 0.7
word1_raw_train, word1_raw_test = train_test_split(word1_raw, train_test_split_ratio)
word2_raw_train, word2_raw_test = train_test_split(word2_raw, train_test_split_ratio)
word3_raw_train, word3_raw_test = train_test_split(word3_raw, train_test_split_ratio)
word4_raw_train, word4_raw_test = train_test_split(word4_raw, train_test_split_ratio)

# Take the same number of readings for all words to be fair
num_samples_train = min(np.shape(word1_raw_train)[0], np.shape(word2_raw_train)[0], np.shape(word3_raw_train)[0], np.shape(word4_raw_train)[0])
word1_raw_train = word1_raw_train[:num_samples_train,:]
word2_raw_train = word2_raw_train[:num_samples_train,:]
word3_raw_train = word3_raw_train[:num_samples_train,:]
word4_raw_train = word4_raw_train[:num_samples_train,:]

Plot your data and get a feel for how it looks enveloped.

**<span style="color:red">Important: It's okay if the data isn't aligned. The code in the next part will automatically align the data.</span>** 

In [ ]:
# Plot all training samples
plt.plot(word1_raw_train.T)
plt.show()
plt.plot(word2_raw_train.T)
plt.show()
plt.plot(word3_raw_train.T)
plt.show()
plt.plot(word4_raw_train.T)
plt.show()

As you can see above, the speech is only a small part of the 2 second window, and each sample starts at different times. PCA is not good at interpreting delay, so we need to somehow start in the same place each time and capture a smaller segment of the 2 second sample where the speech is present. To do this, we will use a thresholding algorithm.

First, we define a **`threshold`** relative to the maximum value of the data. We say that any signal that crosses the threshold is the start of a speech command. In order to not lose the first couple samples of the speech command, we say that the command starts **`pre_length`** samples before the threshold is crossed. We then take a window of the data that is **`length`** long, and try to capture the entire sound of the command in that window.

<b>Play around with the parameters `length`, `pre_length` and `threshold`</b> in the cells below to find appropriate values corresponding to your voice and chosen commands. You should see the results and how much of your command you captured in the plots generated below. When you are satisfied, note down the values of `length`, `pre_length` and `threshold` - <b>you will need to add them to the Launchpad sketch later.</b>

In [ ]:
def get_snippets(data, length, pre_length, thres):
    data_out = np.zeros((np.shape(data)[0], length))
    
    for rnum, row in enumerate(data):
        # Find the threshold
        row_thres = thres*np.max(row)

        # Figure out when interesting snippet starts
        block = pre_length
        while (row[block] < row_thres):
            block = block + 1
        block = min(block, 172 - length)
        data_out[rnum,:] = row[block-pre_length:block-pre_length+length]
        
        # Normalization
        data_out[rnum,:] = data_out[rnum,:] / np.sum(data_out[rnum,:])
        
    return data_out

In [ ]:
length = # Default: 40        # Adjust this
pre_length = # Default: 5     # Adjust this
threshold = # Default:  0.5    # Adjust this

word1_processed_train = get_snippets(word1_raw_train, length, pre_length, threshold)
plt.plot(word1_processed_train.T)
plt.show()
plt.figure()
word2_processed_train = get_snippets(word2_raw_train, length, pre_length, threshold)
plt.plot(word2_processed_train.T)
plt.show()
word3_processed_train = get_snippets(word3_raw_train, length, pre_length, threshold)
plt.plot(word3_processed_train.T)
plt.show()
plt.figure()
word4_processed_train = get_snippets(word4_raw_train, length, pre_length, threshold)
plt.plot(word4_processed_train.T)
plt.show()

You should now see a mostly organized set of samples for each word. Can you tell the which word is which just by the envelope? Can you tell them apart? If you can't tell the words apart, then PCA will have a difficult time as well.

Now that we have our data in a nice format, we can build the PCA input matrix from that data. The function <b>`np.vstack`</b> might be helpful here.


In [ ]:
# YOUR CODE HERE #
processed_A = 

Write code below to perform SVD on your matrix A (is there a function in one of our libraries that can help?), plot the sigma values, and project on to the principal components. First zero-mean your data as `demeaned_A`.

In [ ]:
# Zero-mean the matrix A. Note down the mean for the Launchpad code
# YOUR CODE HERE #
mean_vec = 
demeaned_A = 

In [ ]:
# Take the SVD of matrix demeaned_A
# YOUR CODE HERE #


In [ ]:
# Plot out the sigma values (Hint: Use plt.stem for a stem plot)
# YOUR CODE HERE #

plt.xlim([-1,10])

Take a look at your sigma values. They should show you very clearly how many principal components you need.

**<span style="color:red">How many principal components do you need? Given that you are sorting 4 words, what is the the number you expect to need?</span>** 

There is no correct answer here. We can pick as many principal components onto which we project our data to get the "best" separation (most variance), but at some point, the cost-benefit isn't worth selecting an extra basis vector. For example, in our project, we are loading these basis vectors onto the [MSP430 Launchpad](http://www.ti.com/tool/MSP-EXP430F5529LP), and we can only store 2-3 principal components before we run into memory issues.

In [ ]:
# Plot the principal component(s)
# YOUR CODE HERE #
new_basis =         # This should be the basis containing your principal components
plt.plot(new_basis)

Now project the data in the matrix A onto the new basis and plot it. Do you see clustering? Do you think you can separate the data easily? If not, you might need to try new words.

In [ ]:
# Project the data onto the new basis
# YOUR CODE HERE #
proj = 

plt.scatter(proj[0:num_samples_train,0], proj[0:num_samples_train,1], c=['blue'], edgecolor='none')
plt.scatter(proj[num_samples_train:num_samples_train*2,0], proj[num_samples_train:num_samples_train*2,1], c=['red'], edgecolor='none')
plt.scatter(proj[num_samples_train*2:num_samples_train*3,0], proj[num_samples_train*2:num_samples_train*3,1], c=['green'], edgecolor='none')
plt.scatter(proj[num_samples_train*3:num_samples_train*4,0], proj[num_samples_train*3:num_samples_train*4,1], c=['orange'], edgecolor='none')
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

Your data might look noisy, and might not classify perfectly. That is completely okay, we are just looking for good enough. Like many AI applications, this is noisy data that we are classifying so some error in classification is okay. The important part is if you think that you can see some clustering. 

Once you think you have decent clustering, you can move on to getting your code to automate classification and you will make up for some of the error there, too.

**<span style="color:red">Summary TO DO for Part 2</span>** 
- **<span style="color:red">Load words from CSV</span>** 
- **<span style="color:red">Plot the your enveloped words and make sure they are distinguishable. Else re-record words</span>** 
- **<span style="color:red">Play around with `length`, `prelength` and `threshold` until you settle on desirable values</span>** 
- **<span style="color:red">Construct the A matrix from data</span>** 
- **<span style="color:red">Further process the A matrix (hint hint: De-mean it)</span>** 
- **<span style="color:red">Take the SVD of A and plot the sigma values - how many principal components do you need ? </span>** 
- **<span style="color:red">Project data from A matrix onto selected number of principal components (should you use U or V)</span>** 
- **<span style="color:red">Plot the projections</span>** 
- **<span style="color:red">Make sure that the projected data clusters properly. If clustering is not apparent. re-collect data. </span>** 

<a id='part3'></a>
## <span style="color:blue">Part 3:  Finding Cluster Centers</span>

#### Implement the find_centroids function.

In [ ]:
def find_centroids(clustered_data):
    """ Find the mean of each cluster.
    It may be helpful to recall how you constructed the data matrix (e.g. which rows correspond to which word)
    
    Parameters:
        clustered_data: the data already projected onto the new basis
        
    Returns: 
        The centroids of the clusters
    """
    centroids = [None, None, None, None]
    
    # YOUR CODE HERE #
    
    
    return centroids

In [ ]:
# Determine the centroids of each cluster
# YOUR CODE HERE
centroids = 
print(centroids)

In [ ]:
centroid1 = centroids[0]
centroid2 = centroids[1]
centroid3 = centroids[2]
centroid4 = centroids[3]
centroid_list = np.vstack([centroid1, centroid2, centroid3, centroid4])

print('The first centroid is at: ' + str(centroid1))
print('The second centroid is at: ' + str(centroid2))
print('The third centroid is at: ' + str(centroid3))
print('The fourth centroid is at: ' + str(centroid4))

plt.scatter(proj[0:num_samples_train,0], proj[0:num_samples_train,1], c=['blue'], edgecolor='none')
plt.scatter(proj[num_samples_train:num_samples_train*2,0], proj[num_samples_train:num_samples_train*2,1], c=['red'], edgecolor='none')
plt.scatter(proj[num_samples_train*2:num_samples_train*3,0], proj[num_samples_train*2:num_samples_train*3,1], c=['green'], edgecolor='none')
plt.scatter(proj[num_samples_train*3:num_samples_train*4,0], proj[num_samples_train*3:num_samples_train*4,1], c=['orange'], edgecolor='none')

plt.scatter(centroid_list[:,0], centroid_list[:,1], c=['blue', 'red', 'green', 'orange'], marker='*', s=300)
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Training Data")
plt.show()

<a id='part4'></a>
## <span style="color:blue">Part 4:  Classifying Test Data</span>

Great! We now have the means (centroid) for each word. Now let's see how well our test data performs. Recall that we will classify each data point according to the centroid that it is closest in Euclidean distance to. 

Before we perform classification, we need to do the same preprocessing to the test data that we did to the training data (enveloping, demeaning, projecting onto the PCA basis). You have already written most of the code for this part. However, note the difference in variable names as we are now working with test data.

First let's look at what our raw test data looks like.

In [ ]:
# Take the same number of readings for all words to be fair
num_samples_test = min(np.shape(word1_raw_test)[0], np.shape(word2_raw_test)[0], np.shape(word3_raw_test)[0], np.shape(word4_raw_test)[0])
word1_raw_test = word1_raw_test[:num_samples_test,:]
word2_raw_test = word2_raw_test[:num_samples_test,:]
word3_raw_test = word3_raw_test[:num_samples_test,:]
word4_raw_test = word4_raw_test[:num_samples_test,:]

In [ ]:
# Plot all test samples
plt.plot(word1_raw_test.T)
plt.show()
plt.plot(word2_raw_test.T)
plt.show()
plt.plot(word3_raw_test.T)
plt.show()
plt.plot(word4_raw_test.T)
plt.show()

Perform enveloping and trimming of our test data.

In [ ]:
length = # Default: 40        # Adjust this
pre_length = # Default: 5     # Adjust this
threshold = # Default:  0.5    # Adjust this

word1_processed_test = get_snippets(word1_raw_test, length, pre_length, threshold)
plt.plot(word1_processed_test.T)
plt.show()
plt.figure()
word2_processed_test = get_snippets(word2_raw_test, length, pre_length, threshold)
plt.plot(word2_processed_test.T)
plt.show()
word3_processed_test = get_snippets(word3_raw_test, length, pre_length, threshold)
plt.plot(word3_processed_test.T)
plt.show()
plt.figure()
word4_processed_test = get_snippets(word4_raw_test, length, pre_length, threshold)
plt.plot(word4_processed_test.T)
plt.show()

Construct the PCA matrix by stacking all the test data.

In [ ]:
# YOUR CODE HERE #
processed_A_test = 

Zero-mean the test data using the **same mean vector** as the one computed with the training data.

In [ ]:
# YOUR CODE HERE #
processed_A_test = 

Project the test data onto the **same PCA basis** as the one computed with the training data. Plot them to see how well your test data clusters in this new basis. This will give you an idea of how well your test data will classify.

In [ ]:
# YOUR CODE HERE #
proj = 

plt.scatter(proj[0:num_samples_test,0], proj[0:num_samples_test,1], c=['blue'], edgecolor='none')
plt.scatter(proj[num_samples_test:num_samples_test*2,0], proj[num_samples_test:num_samples_test*2,1], c=['red'], edgecolor='none')
plt.scatter(proj[num_samples_test*2:num_samples_test*3,0], proj[num_samples_test*2:num_samples_test*3,1], c=['green'], edgecolor='none')
plt.scatter(proj[num_samples_test*3:num_samples_test*4,0], proj[num_samples_test*3:num_samples_test*4,1], c=['orange'], edgecolor='none')
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))

plt.scatter(centroid_list[:,0], centroid_list[:,1], c=['blue', 'red', 'green', 'orange'], marker='*', s=300)
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Test Data")
plt.show()

Now that we have some idea of how our test data looks in this new basis, let's see how our data actually performs. Implement the classify function that takes in a data point (AFTER enveloping and demeaning are applied) and returns which word number it belongs to depending on which centroid the data point is closest in Euclidean distance to. 

In [ ]:
def classify(data_point):
    """
    Classifies a new reading vector into a word.
    Inputs:
        data_point: new data point vector (before projection)
    Output:
        Word number (should be in {1, 2, 3, 4})
    """
    # YOUR CODE HERE
    

In [ ]:
# Try out the classification function
print(classify(processed_A_test[0,:])) # Modify to use other vectors

**Our goal is 80% accuracy for each word.** Write code to apply the `classify` function to each sample and compute the accuracy for each word.

In [ ]:
# Try to classify the whole A matrix
correct_counts = np.zeros(4)

for (row_num, data) in enumerate(processed_A_test):
    # YOUR CODE HERE

        
for i in range(len(correct_counts)):
    print("Percent correct of word {} = {}%".format(i+1, 100*correct_counts[i]/num_samples_test))

**<span style="color:red">Summary TO DO for Part 4:</span>** 
- **<span style="color:red">Fill in function to classify the words - make sure you apply the same preprocessing to the words (demeaning)</span>** 
- **<span style="color:red">Make sure you have an 80% classification rate for each word. Else you need to re-collect data</span>** 